# I used the sift+flann approach with feature matching and homography to find objects

In [32]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [33]:
def detect_edges(filename_in: str, filename_out: str):
    img = cv2.imread(filename_in)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, 0.04)
    dst = cv2.dilate(dst, None)
    img[dst > 0.01 * dst.max()] = [0, 0, 255]

    cv2.imwrite(filename_out, img)

In [34]:
def align_img(image_to_align: str, reference_image: str, max_features, good_match_percent):
    MIN_NUM_MATCHES = 11

    # Load images
    img_ref = cv2.imread(reference_image, cv2.IMREAD_GRAYSCALE)
    img_src = cv2.imread(image_to_align, cv2.IMREAD_GRAYSCALE)

    sift = cv2.SIFT_create(nfeatures=max_features)
    kp1, des1 = sift.detectAndCompute(img_ref, None)
    kp2, des2 = sift.detectAndCompute(img_src, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)
    good = [m for m, n in matches if m.distance < good_match_percent * n.distance]

    if len(good) >= MIN_NUM_MATCHES:
        src_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
        h, w = img_ref.shape
        aligned = cv2.warpPerspective(cv2.imread(image_to_align), H, (w, h))
        cv2.imwrite("out/aligned.png", aligned)
    else:
        print(f"Not enough matches found: {len(good)}/{MIN_NUM_MATCHES}")
        matches_mask = None

    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=None,
                       matchesMask=matches_mask,
                       flags=2)
    
    img3 = cv2.drawMatches(img_ref, kp1, img_src, kp2, good, None, **draw_params)

    cv2.imwrite('out/matches.png', img3)

In [35]:
detect_edges('reference_img.png', 'out/harris.png')
align_img('align_this.jpg', 'reference_img.png', 5_000_000, 0.7)

In [36]:
from PIL import Image

image_paths = ['out/harris.png', 'out/matches.png', 'out/aligned.png']
images = [Image.open(p).convert("RGB") for p in image_paths]
if images:
    images[0].save("out/results.pdf", save_all=True, append_images=images[1:])
